In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
car = pd.read_csv("second_cars_info.csv")


In [2]:
#============== 查询引擎 ==============#

# 加载数据

# Brand
brand = input("请输入品牌名称:")
car = car[car['Brand'].str.contains(brand)]

# Name
name = input("请输入车名：")
car = car[car['Name'].str.contains(name)]

# Boarding_time
boarding_time = input("请输入上牌时间范围（例如2005年8月-2010年10月）或输入‘未上牌’：")
if boarding_time == '未上牌':
    car = car[car['Boarding_time'] == '未上牌']
elif boarding_time != '':
    car = car[car['Boarding_time'] != '未上牌'] # 删除未上牌的数据，目的是后面能让Boarding_time这一列变成datatime类型,从而可以比较大小
    car['Boarding_time'] = pd.to_datetime(car['Boarding_time'], format='%Y年%m月')
    bodTime_lower = datetime.strptime(boarding_time.split('-')[0], '%Y年%m月')
    bodTime_upper = datetime.strptime(boarding_time.split('-')[1], '%Y年%m月')
    car = car[(car['Boarding_time'] >= bodTime_lower) & (car['Boarding_time'] <= bodTime_upper)]

# Km
distance = input("请输入行驶里程范围（单位：万公里）（例如3.5-10.0）：")
if distance == '百公里内':
    car = car[car['Km'] == '百公里内']
elif distance != '':
    car = car = car[car['Km'] != '百公里内'] # 删除百公里内的数据，目的是后面能让Km这一列变成float类型，从而可以比较大小
    dist_numeric = pd.to_numeric(car.loc[:,'Km'].apply(lambda x : re.search('\d+\.\d+', x).group())) # 将剩余Km列的数据变为float
    dist_lower,dist_upper = pd.to_numeric(distance.split("-"))
    car = car[(dist_numeric >= dist_lower) & (dist_numeric <= dist_upper)]

# Discharge
discharge = input("请输入Discharge：")
car = car[car['Discharge'].str.contains(name)]

# Sec_price
sec_price = input("请输入二手价范围（单位：万）（例如3.5-10.0）：")
if sec_price != '':
    secPrice_lower,secPrice_upper = pd.to_numeric(sec_price.split("-"))
    car['Sec_price'] = pd.to_numeric(car['Sec_price']) # 将Sec_price变为数字类型
    car = car[(car['Sec_price'] >= secPrice_lower) & (car['Sec_price'] <= secPrice_upper)]

# New_price
new_price = input("请输入全新价范围（单位：万）（例如3.5-10.0）:")
if new_price != '':
    newPrice_lower,newPrice_upper = pd.to_numeric(new_price.split("-"))
    price_numeric = pd.to_numeric(car.loc[:,'New_price'].apply(lambda x : re.search('\d+\.\d+', x).group()))
    car = car[(price_numeric >= newPrice_lower) & (price_numeric <= newPrice_upper)]

car

,Brand,Name,Boarding_time,Km,Discharge,Sec_price,New_price
0,奥迪,奥迪A6L 2006款 2.4 CVT 舒适型,2006年8月,9.00万公里,国3,6.90,50.89万
1,奥迪,奥迪A6L 2007款 2.4 CVT 舒适型,2007年1月,8.00万公里,国4,8.88,50.89万
2,奥迪,奥迪A6L 2004款 2.4L 技术领先型,2005年5月,15.00万公里,国2,3.82,54.24万
3,奥迪,奥迪A8L 2013款 45 TFSI quattro舒适型,2013年10月,4.80万公里,欧4,44.80,101.06万
4,奥迪,奥迪A6L 2014款 30 FSI 豪华型,2014年9月,0.81万公里,"国4,国5",33.19,54.99万
...,...,...,...,...,...,...,...
753,奥迪,奥迪A6L 2016款 TFSI 运动型,2016年2月,0.74万公里,国5,34.00,50.06万
754,奥迪,奥迪A5 2010款 Cabriolet 2.0T,2011年8月,7.00万公里,欧4,23.88,64.91万
755,奥迪,奥迪A5 2010款 Sportback 2.0T 豪华型,2010年12月,8.00万公里,欧4,19.50,62.74万
756,奥迪,奥迪A5 2010款 Sportback 2.0T 舒适型,2011年5月,8.00万公里,欧4,18.00,53.61万


In [12]:
car.loc[car['Brand'].str.contains('奥迪')]

,Brand,Name,Boarding_time,Km,Discharge,Sec_price,New_price
0,奥迪,奥迪A6L 2006款 2.4 CVT 舒适型,2006年8月,9.00万公里,国3,6.90,50.89万
1,奥迪,奥迪A6L 2007款 2.4 CVT 舒适型,2007年1月,8.00万公里,国4,8.88,50.89万
2,奥迪,奥迪A6L 2004款 2.4L 技术领先型,2005年5月,15.00万公里,国2,3.82,54.24万
3,奥迪,奥迪A8L 2013款 45 TFSI quattro舒适型,2013年10月,4.80万公里,欧4,44.80,101.06万
4,奥迪,奥迪A6L 2014款 30 FSI 豪华型,2014年9月,0.81万公里,"国4,国5",33.19,54.99万
...,...,...,...,...,...,...,...
753,奥迪,奥迪A6L 2016款 TFSI 运动型,2016年2月,0.74万公里,国5,34.00,50.06万
754,奥迪,奥迪A5 2010款 Cabriolet 2.0T,2011年8月,7.00万公里,欧4,23.88,64.91万
755,奥迪,奥迪A5 2010款 Sportback 2.0T 豪华型,2010年12月,8.00万公里,欧4,19.50,62.74万
756,奥迪,奥迪A5 2010款 Sportback 2.0T 舒适型,2011年5月,8.00万公里,欧4,18.00,53.61万
